# Examples by Use Case

本文将详细介绍如何使用 ``aws_ssm_run_command`` 库来实现各种常见的 Use Cases.

## Prepare

In [3]:
import uuid
import json
from pathlib import Path

from rich import print as rprint
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context

import aws_ssm_run_command.api as aws_ssm_run_command

# set boto session
bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")
# set two instance ids
inst_id_1 = "i-00d17e6620f53ea14"
inst_id_2 = "i-04263cc722e9b0ac3"
# set an S3 bucket location for storing the output
s3dir = S3Path(f"s3://{bsm.aws_account_alias}-{bsm.aws_region}-data/projects/aws_ssm_run_command/example/")
# set the current working directory
dir_here = Path.cwd().absolute()
# tell s3pathlib to use the given boto session
context.attach_boto_session(bsm.boto_ses)

In [4]:
def rprint_response(res):
    """
    A helper function to print the response of boto3 API.
    """
    if "ResponseMetadata" in res:
        del res["ResponseMetadata"]
    rprint(res)
    
def s3url_to_s3uri(url: str) -> str:
    """
    Convert https://s3.amazonaws.com/bucket/key to s3://bucket/key 
    """
    return "s3://" + url.split("/", 3)[-1]

In [37]:
value = uuid.uuid4()
cmd = f"echo {value} > ~/uuid.txt"
print("--- uuid value ---")
rprint(value)

print("--- command ---")
rprint(cmd)

--- uuid value ---


4441c17d-08a0-4156-b6ef-13364fbeb4e6

--- command ---


echo 4441c17d-08a0-4156-b6ef-13364fbeb4e6 > ~/uuid.txt

In [6]:
print("--- send command and wait it to succeed ---")
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=cmd,
    instance_ids=inst_id_1,
)

--- send command and wait it to succeed ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...


In [7]:
print("--- command invocation details ---")
rprint(command_invocations)

--- command invocation details ---


[
    CommandInvocation(
        CommandId='599ed08c-45c3-4267-ade3-df3712dcc024',
        InstanceId='i-00d17e6620f53ea14',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:38.143Z',
        ExecutionElapsedTime='PT0.006S',
        ExecutionEndDateTime='2024-06-18T03:53:38.143Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='',
        StandardOutputUrl='',
        StandardErrorContent='',
        StandardErrorUrl='',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    )
]

In [8]:
print("--- send another command and wait it to succeed ---")
cmd = "cat ~/uuid.txt"
rprint(cmd)
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=cmd,
    instance_ids=inst_id_1,
)

--- send another command and wait it to succeed ---


cat ~/uuid.txt

start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...


In [9]:
print("--- stdout ---")
stdout = command_invocations[0].StandardOutputContent.strip()
rprint(f"stdout = {stdout}, uuid = {value}")

--- stdout ---


stdout = f2266cf8-96ed-46b3-a07f-2eff5c6a1d85, uuid = f2266cf8-96ed-46b3-a07f-2eff5c6a1d85

In [10]:
print("--- command ---")
cmd = "python3 --version"
rprint(cmd)

--- command ---


python3 --version

In [11]:
print("--- send command and wait it to succeed ---")
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=cmd,
    instance_ids=inst_id_1,
)

--- send command and wait it to succeed ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...


In [12]:
print("--- command invocation details ---")
rprint(command_invocations)

--- command invocation details ---


[
    CommandInvocation(
        CommandId='ebf0307f-a271-4118-9a3b-265a8fb4b75d',
        InstanceId='i-00d17e6620f53ea14',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:40.564Z',
        ExecutionElapsedTime='PT0.006S',
        ExecutionEndDateTime='2024-06-18T03:53:40.564Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='Python 3.9.16\n',
        StandardOutputUrl='',
        StandardErrorContent='',
        StandardErrorUrl='',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    )
]

In [13]:
print("--- stdout ---")
rprint(command_invocations[0].StandardOutputContent.strip())

--- stdout ---


Python 3.9.16

In [14]:
# clean up the s3 folder to ensure a fresh start
s3dir.delete()

S3Path('s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/')

In [15]:
print("--- command ---")
cmd = "python3 --version"
rprint(cmd)

--- command ---


python3 --version

In [16]:
print("--- send command and wait it to succeed ---")
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=cmd,
    instance_ids=inst_id_1,
    output_s3_bucket_name=s3dir.bucket,
    output_s3_key_prefix=s3dir.key,
)

--- send command and wait it to succeed ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...


In [17]:
print("--- command invocation details ---")
rprint(command_invocations)

--- command invocation details ---


[
    CommandInvocation(
        CommandId='88f5b32a-365e-4f26-a7d2-c65d48dbfac1',
        InstanceId='i-00d17e6620f53ea14',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:42.065Z',
        ExecutionElapsedTime='PT0.105S',
        ExecutionEndDateTime='2024-06-18T03:53:42.065Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='Python 3.9.16\n',
        StandardOutputUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comma
nd/example/88f5b32a-365e-4f26-a7d2-c65d48dbfac1/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout',
        StandardErrorContent='',
        StandardErrorUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comman
d/example/88f5b32a-365e-4f26-a7d2-c65d48dbfac1/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stderr',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    )
]

In [18]:
print("--- s3 dir files ---")
for s3path in s3dir.iter_objects():
    rprint(s3path.uri)

--- s3 dir files ---


s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/88f5b32a-365e-4f26-a7d2-c65d48dbfac1/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout

In [19]:
s3path = S3Path(s3url_to_s3uri(command_invocations[0].StandardOutputUrl))
rprint(f"read stdout from {s3path.uri}:")
print("--- stdout ---")
rprint(s3path.read_text().strip())

read stdout from 
s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/88f5b32a-365e-4f26-a7d2-c65d48dbfac1/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout:

--- stdout ---


Python 3.9.16

In [20]:
# clean up the s3 folder to ensure a fresh start
s3dir.delete()

S3Path('s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/')

In [21]:
print("--- command ---")
commands = [
    "aws --version",
    "python3 --version",
]
rprint(commands)

--- command ---


['aws --version', 'python3 --version']

In [22]:
print("--- send command and wait it to succeed ---")
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=commands,
    instance_ids=[inst_id_1, inst_id_2],
    output_s3_bucket_name=s3dir.bucket,
    output_s3_key_prefix=s3dir.key,
)

--- send command and wait it to succeed ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 1 th attempt, elapsed 3 seconds, remain 57 seconds ...
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...


In [23]:
print("--- command invocation details ---")
rprint(command_invocations)

--- command invocation details ---


[
    CommandInvocation(
        CommandId='a4621bd0-192f-41f5-a891-88a5d91eeeac',
        InstanceId='i-00d17e6620f53ea14',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:43.547Z',
        ExecutionElapsedTime='PT0.62S',
        ExecutionEndDateTime='2024-06-18T03:53:43.547Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='aws-cli/2.15.30 Python/3.9.16 Linux/6.1.92-99.174.amzn2023.x86_64 
source/x86_64.amzn.2023 prompt/off\nPython 3.9.16\n',
        StandardOutputUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comma
nd/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout',
        StandardErrorContent='',
        StandardErrorUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comman
d/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stderr',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    ),
    CommandInvocation(
        CommandId='a4621bd0-192f-41f5-a891-88a5d91eeeac',
        InstanceId='i-04263cc722e9b0ac3',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:43.563Z',
        ExecutionElapsedTime='PT0.623S',
        ExecutionEndDateTime='2024-06-18T03:53:43.563Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='aws-cli/2.15.30 Python/3.9.16 Linux/6.1.92-99.174.amzn2023.x86_64 
source/x86_64.amzn.2023 prompt/off\nPython 3.9.16\n',
        StandardOutputUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comma
nd/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-04263cc722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stdout',
        StandardErrorContent='',
        StandardErrorUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comman
d/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-04263cc722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stderr',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    )
]

In [24]:
print("--- s3 dir files ---")
for s3path in s3dir.iter_objects():
    rprint(s3path.uri)

--- s3 dir files ---


s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout

s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-04263cc
722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stdout

In [25]:
for command_invocation in command_invocations:
    s3path = S3Path(s3url_to_s3uri(command_invocation.StandardOutputUrl))
    rprint(f"read stdout from {s3path.uri}:")
    print("--- stdout ---")
    rprint(s3path.read_text().strip())

read stdout from 
s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout:

--- stdout ---


aws-cli/2.15.30 Python/3.9.16 Linux/6.1.92-99.174.amzn2023.x86_64 source/x86_64.amzn.2023 prompt/off
Python 3.9.16

read stdout from 
s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/a4621bd0-192f-41f5-a891-88a5d91eeeac/i-04263cc
722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stdout:

--- stdout ---


aws-cli/2.15.30 Python/3.9.16 Linux/6.1.92-99.174.amzn2023.x86_64 source/x86_64.amzn.2023 prompt/off
Python 3.9.16

In [26]:
# clean up the s3 folder to ensure a fresh start
s3dir.delete()

S3Path('s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/')

In [27]:
print("--- command ---")
commands = [
    "python3 --version", # This always work
    # I have run "sudo python3 -m ensurepip --upgrade" in instance 1 already
    "pip3 --version", # This only works on instance 1, instance 2 doesn't have "pip"
]
rprint(commands)

--- command ---


['python3 --version', 'pip3 --version']

In [28]:
print("--- send command and wait it to succeed ---")
command_invocations = aws_ssm_run_command.better_boto.run_shell_script_sync(
    ssm_client=bsm.ssm_client,
    commands=commands,
    instance_ids=[inst_id_1, inst_id_2],
    output_s3_bucket_name=s3dir.bucket,
    output_s3_key_prefix=s3dir.key,
    allow_fails_config=2,
)

--- send command and wait it to succeed ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 0 th attempt, elapsed 0 seconds, remain 60 seconds ...

In [29]:
print("--- command invocation details ---")
rprint(command_invocations)

--- command invocation details ---


[
    CommandInvocation(
        CommandId='e6101c42-ba78-48f8-ac2f-56eb90913579',
        InstanceId='i-00d17e6620f53ea14',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=0,
        ExecutionStartDateTime='2024-06-18T03:53:48.217Z',
        ExecutionElapsedTime='PT0.385S',
        ExecutionEndDateTime='2024-06-18T03:53:48.217Z',
        Status='Success',
        StatusDetails='Success',
        StandardOutputContent='Python 3.9.16\npip 21.3.1 from /usr/local/lib/python3.9/site-packages/pip (python 
3.9)\n',
        StandardOutputUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comma
nd/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout',
        StandardErrorContent='',
        StandardErrorUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comman
d/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-00d17e6620f53ea14/awsrunShellScript/0.awsrunShellScript/stderr',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    ),
    CommandInvocation(
        CommandId='e6101c42-ba78-48f8-ac2f-56eb90913579',
        InstanceId='i-04263cc722e9b0ac3',
        Comment='',
        DocumentName='AWS-RunShellScript',
        DocumentVersion='1',
        PluginName='aws:runShellScript',
        ResponseCode=127,
        ExecutionStartDateTime='2024-06-18T03:53:48.235Z',
        ExecutionElapsedTime='PT0.122S',
        ExecutionEndDateTime='2024-06-18T03:53:48.235Z',
        Status='Failed',
        StatusDetails='Failed',
        StandardOutputContent='Python 3.9.16\n',
        StandardOutputUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comma
nd/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-04263cc722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stdout',
        StandardErrorContent='/var/lib/amazon/ssm/i-04263cc722e9b0ac3/document/orchestration/e6101c42-ba78-48f8-ac2
f-56eb90913579/awsrunShellScript/0.awsrunShellScript/_script.sh: line 2: pip3: command not found\nfailed to run 
commands: exit status 127',
        StandardErrorUrl='https://s3.us-east-1.amazonaws.com/bmt-app-dev-us-east-1-data/projects/aws_ssm_run_comman
d/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-04263cc722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stderr',
        CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
    )
]

In [30]:
print("--- s3 dir files ---")
for s3path in s3dir.iter_objects():
    rprint(s3path.uri)

--- s3 dir files ---


s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout

s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-04263cc
722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stderr

s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-04263cc
722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stdout

In [31]:
for command_invocation in command_invocations:
    print(f"=== instance id = {command_invocation.InstanceId} ===")
    if command_invocation.is_success():
        s3path = S3Path(s3url_to_s3uri(command_invocation.StandardOutputUrl))
        rprint(f"read stdout from {s3path.uri}:")
        rprint("--- stdout --- ")
        rprint(s3path.read_text().strip())
    else:
        s3path = S3Path(s3url_to_s3uri(command_invocation.StandardOutputUrl))
        rprint(f"read stdout from {s3path.uri}:")
        rprint("--- stdout --- ")
        rprint(s3path.read_text().strip())
        
        s3path = S3Path(s3url_to_s3uri(command_invocation.StandardErrorUrl))
        rprint(f"read stderr from {s3path.uri}:")
        rprint("--- stderr --- ")
        rprint(s3path.read_text().strip())

=== instance id = i-00d17e6620f53ea14 ===


read stdout from 
s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-00d17e6
620f53ea14/awsrunShellScript/0.awsrunShellScript/stdout:

--- stdout ---

Python 3.9.16
pip 21.3.1 from /usr/local/lib/python3.9/site-packages/pip (python 3.9)

=== instance id = i-04263cc722e9b0ac3 ===


...

--- stdout ---

Python 3.9.16

read stderr from 
s3://bmt-app-dev-us-east-1-data/projects/aws_ssm_run_command/example/e6101c42-ba78-48f8-ac2f-56eb90913579/i-04263cc
722e9b0ac3/awsrunShellScript/0.awsrunShellScript/stderr:

--- stderr ---

/var/lib/amazon/ssm/i-04263cc722e9b0ac3/document/orchestration/e6101c42-ba78-48f8-ac2f-56eb90913579/awsrunShellScri
pt/0.awsrunShellScript/_script.sh: line 2: pip3: command not found
failed to run commands: exit status 127

In [32]:
path_script = dir_here.joinpath("script.py")
print("--- script source code ---")
rprint(path_script.read_text())

--- script source code ---


# -*- coding: utf-8 -*-

import sys
import json


def run() -> dict:
    print("start")
    print("done")
    return {
        "python": sys.executable,
        "weird_string": "\\a\nb\tc\"d'e@f#g:h/i",
    }


if __name__ == "__main__":
    print(json.dumps(run()))

In [33]:
path_aws = "aws"
path_python = "python3"
code = path_script.read_text()
s3uri = s3dir.joinpath("script.py").uri
args = []

print("--- command invocation ---")
command_invocation = aws_ssm_run_command.patterns.run_command_on_one_ec2.run_python_script(
    ssm_client=bsm.ssm_client,
    s3_client=bsm.s3_client,
    instance_id=inst_id_1,
    path_aws=path_aws, # what is the path to the aws cli executable 
    path_python=path_python, # what is the path to the Python executable 
    code=code, # source code
    s3uri=s3uri, # where you want to upload your code to on S3?
    args=args, # additional argument for ``python your_script.py arg1 arg2 ...``
)
rprint(command_invocation)

--- command invocation ---
start waiter, polling every 3 seconds, timeout in 60 seconds.
on 1 th attempt, elapsed 3 seconds, remain 57 seconds ...


CommandInvocation(
    CommandId='cf85b5bf-c80d-455e-9bdc-2fca51dd7b5d',
    InstanceId='i-00d17e6620f53ea14',
    Comment='',
    DocumentName='AWS-RunShellScript',
    DocumentVersion='1',
    PluginName='aws:runShellScript',
    ResponseCode=0,
    ExecutionStartDateTime='2024-06-18T03:53:49.876Z',
    ExecutionElapsedTime='PT0.849S',
    ExecutionEndDateTime='2024-06-18T03:53:49.876Z',
    Status='Success',
    StatusDetails='Success',
    StandardOutputContent='start\ndone\n{"python": "/usr/bin/python3", "weird_string": 
"\\\\a\\nb\\tc\\"d\'e@f#g:h/i"}\n',
    StandardOutputUrl='',
    StandardErrorContent='',
    StandardErrorUrl='',
    CloudWatchOutputConfig={'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}
)

In [34]:
output_lines = command_invocation.StandardOutputContent.split("\n")
print("--- print function output ---")
rprint(output_lines[:2])

--- print function output ---


['start', 'done']

In [35]:
print("--- python function return value ---")
data = json.loads(output_lines[2])
rprint(data)
print("--- python value ---")
rprint(data["python"])
print("--- weird_string value ---")
rprint(data["weird_string"]) # test the string escape

--- python function return value ---


{'python': '/usr/bin/python3', 'weird_string': '\\a\nb\tc"d\'e@f#g:h/i'}

--- python value ---


/usr/bin/python3

--- weird_string value ---


\a
b       c"d'e@f#g:h/i